In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = "../"

import pandas as pd

In [23]:
snp_genes_df = pd.read_csv(home_dir+"data/dbsnp/parsed/genes.tsv", sep="\t", index_col=False) # index_col=False is a must
snp_genes_df.drop_duplicates(keep="first", inplace=True, ignore_index=True)
print(snp_genes_df.shape)
def print_number_of_genes_attached(snps_df):
    merged_df = pd.merge(left=snps_df, right=snp_genes_df, left_on="SNP_ID", right_on="SNP_ID", how="inner")
    print("#-genes:", merged_df["GENE_NAME"].unique().shape[0])

(8242603, 3)


In [2]:
snps_filepath = home_dir+"data/dbsnp/processed/snps_filtered_dupSNPIds_noNPVars.tsv"
snps_df = pd.read_csv(snps_filepath, sep="\t")
print(snps_df.columns)
print(snps_df.shape)
print(snps_df["SNP_ID"].unique().shape)

/tmp/ipykernel_2858092/2977213971.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  snps_df = pd.read_csv(snps_filepath, sep="\t")


Index(['SNP_ID', 'SNP_CLASS', 'CLINICAL_SIGNIFICANCE', 'ACC', 'CHR', 'CHRPOS',
       'SPDI', 'FXN_CLASS', 'VALIDATED', 'TAX_ID', 'CREATEDATE', 'UPDATEDATE',
       'DOCSUM', 'NP_VARIANTS', 'REF_ALLELE', 'ALT_ALLELES'],
      dtype='object')
(6445805, 16)
(6445805,)


In [4]:
rs_ids = snps_df["SNP_ID"].unique().tolist()
len(rs_ids)

6445805

### Reading and filtering ALFA data as it is big

In [5]:
# do not run this unless you changed something before, it takes 1 hour
# better use the following block
# Variants reported on GRCh38, readme: https://www.ncbi.nlm.nih.gov/snp/docs/gsr/ftp_help/
from utils.column_names_reader import get_col_names_from_gzip
filepath = home_dir+"data/ALFA/freq.vcf.gz" # 11GB
col_names = get_col_names_from_gzip(filepath, "#CHROM")
alfa_df_iterator = pd.read_csv(filepath, compression='gzip', chunksize=1000000, comment="#", names=col_names, delim_whitespace=False, sep="\t")
# alfa_df_iterator.__next__() # first df 

chunk_list = []
for i, alfa_df in enumerate(alfa_df_iterator):
    chunk = alfa_df[alfa_df["ID"].isin(rs_ids)]
    chunk_list.append(chunk)
    print(i, chunk.shape)
    # if i==0: break

alfa_filteredOn_rsIds_df = pd.concat(chunk_list)  
print(alfa_filteredOn_rsIds_df.shape)
print(alfa_filteredOn_rsIds_df.columns)
alfa_filteredOn_rsIds_df

0 (20320, 21)
1 (5357, 21)
2 (9182, 21)
3 (13917, 21)
4 (11011, 21)
5 (12917, 21)
6 (13226, 21)
7 (9130, 21)
8 (14053, 21)
9 (6290, 21)
10 (10626, 21)
11 (9439, 21)
12 (11429, 21)
13 (14759, 21)
14 (9378, 21)
15 (2688, 21)
16 (11362, 21)
17 (2438, 21)
18 (2131, 21)
19 (5754, 21)
20 (3994, 21)
21 (1688, 21)
22 (3152, 21)
23 (3951, 21)
24 (362, 21)
25 (3520, 21)
26 (3390, 21)
27 (5915, 21)
28 (6969, 21)
29 (660, 21)
30 (4308, 21)
31 (1853, 21)
32 (712, 21)
33 (13166, 21)
34 (7724, 21)
35 (6079, 21)
36 (4834, 21)
37 (0, 21)
38 (0, 21)
39 (5085, 21)
40 (9436, 21)
41 (25134, 21)
42 (19850, 21)
43 (18164, 21)
44 (13949, 21)
45 (2037, 21)
46 (4592, 21)
47 (8254, 21)
48 (4646, 21)
49 (4596, 21)
50 (7416, 21)
51 (6523, 21)
52 (3528, 21)
53 (450, 21)
54 (1067, 21)
55 (5560, 21)
56 (9504, 21)
57 (12103, 21)
58 (9353, 21)
59 (4398, 21)
60 (4574, 21)
61 (3754, 21)
62 (3498, 21)
63 (7652, 21)
64 (12898, 21)
65 (6042, 21)
66 (5255, 21)
67 (7234, 21)
68 (3126, 21)
69 (2811, 21)
70 (10055, 21)
71 (6029

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMN10492695,...,SAMN10492697,SAMN10492698,SAMN10492699,SAMN10492700,SAMN10492701,SAMN10492702,SAMN11605645,SAMN10492703,SAMN10492704,SAMN10492705
14395,NC_000001.11,69071,rs1314111151,C,"A,G",.,.,.,AN:AC,"9690:0,0",...,"86:0,0","2784:0,0","146:0,0","610:0,0","26:0,0","98:0,0","496:0,0","2898:0,0","112:0,0","14050:0,0"
14400,NC_000001.11,69088,rs1429062358,T,C,.,.,.,AN:AC,9690:1,...,86:0,2784:0,146:0,610:0,26:0,98:0,496:0,2898:0,112:0,14050:1
14402,NC_000001.11,69092,rs985675606,T,C,.,.,.,AN:AC,7618:0,...,84:0,2708:0,146:0,610:0,24:0,94:0,470:0,2816:0,108:0,11862:0
14405,NC_000001.11,69116,rs757299236,G,A,.,.,.,AN:AC,7618:0,...,84:0,2708:0,146:0,610:0,24:0,94:0,470:1,2816:0,108:0,11862:1
14410,NC_000001.11,69134,rs781394307,A,G,.,.,.,AN:AC,7618:9,...,84:0,2708:0,146:0,610:5,24:0,94:0,470:0,2816:0,108:0,11862:14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904166908,NW_009646206.1,181334,rs201445440,G,A,.,.,.,AN:AC,130266:3,...,2560:0,798:0,290:0,336:0,600:0,176:0,5190:0,852:0,3160:0,140270:3
904166911,NW_009646206.1,181361,rs34118903,T,C,.,.,.,AN:AC,155924:1,...,2592:0,1488:8,646:4,358:0,630:0,176:0,11750:4,1550:8,3222:0,173626:17
904166912,NW_009646206.1,181365,rs77312595,G,A,.,.,.,AN:AC,150974:197,...,4400:0,1790:3,660:0,392:1,1800:0,192:0,15668:16,1852:3,6200:0,175938:217
904166913,NW_009646206.1,181377,rs782208370,C,T,.,.,.,AN:AC,16802:2,...,0:0,20:0,348:0,18:0,0:0,0:0,4056:0,20:0,0:0,21244:2


In [7]:
filepath = home_dir+"data/ALFA/alfa_filteredOn_rsIds.tsv" # ?GB
# alfa_filteredOn_rsIds_df.to_csv(filepath, sep="\t", index=False)
alfa_filteredOn_rsIds_df = pd.read_csv(filepath, sep="\t")
print(alfa_filteredOn_rsIds_df.columns)
print(alfa_filteredOn_rsIds_df.shape)
print(alfa_filteredOn_rsIds_df["ID"].unique().shape)

Index(['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT',
       'SAMN10492695', 'SAMN10492696', 'SAMN10492697', 'SAMN10492698',
       'SAMN10492699', 'SAMN10492700', 'SAMN10492701', 'SAMN10492702',
       'SAMN11605645', 'SAMN10492703', 'SAMN10492704', 'SAMN10492705'],
      dtype='object')
(5072966, 21)
(5072966,)


In [8]:
def is_all_alt_alleles_popu_count_zero(row):
    # print(row)
    # raise
    mt_popu_counts = row.SAMN10492705.split(":")[1].split(",")
    for mt_popu_count in mt_popu_counts:
        mt_popu_count = int(mt_popu_count)
        if mt_popu_count != 0:
            return False
    return True

alfa_filteredOn_rsIds_allAltAlleles0_df = alfa_filteredOn_rsIds_df[alfa_filteredOn_rsIds_df.apply(is_all_alt_alleles_popu_count_zero, axis=1)]
print(alfa_filteredOn_rsIds_allAltAlleles0_df.shape)
alfa_filteredOn_rsIds_allAltAlleles0_df

(3336759, 21)


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMN10492695,...,SAMN10492697,SAMN10492698,SAMN10492699,SAMN10492700,SAMN10492701,SAMN10492702,SAMN11605645,SAMN10492703,SAMN10492704,SAMN10492705
0,NC_000001.11,69071,rs1314111151,C,"A,G",.,.,.,AN:AC,"9690:0,0",...,"86:0,0","2784:0,0","146:0,0","610:0,0","26:0,0","98:0,0","496:0,0","2898:0,0","112:0,0","14050:0,0"
2,NC_000001.11,69092,rs985675606,T,C,.,.,.,AN:AC,7618:0,...,84:0,2708:0,146:0,610:0,24:0,94:0,470:0,2816:0,108:0,11862:0
6,NC_000001.11,69184,rs1231643173,G,A,.,.,.,AN:AC,7618:0,...,84:0,2708:0,146:0,610:0,24:0,94:0,470:0,2816:0,108:0,11862:0
8,NC_000001.11,69217,rs1465147285,G,T,.,.,.,AN:AC,7618:0,...,84:0,2708:0,146:0,610:0,24:0,94:0,470:0,2816:0,108:0,11862:0
9,NC_000001.11,69218,rs746046455,T,C,.,.,.,AN:AC,7618:0,...,84:0,2708:0,146:0,610:0,24:0,94:0,470:0,2816:0,108:0,11862:0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5072893,NW_009646206.1,177535,rs139614688,C,A,.,.,.,AN:AC,6148:0,...,0:0,0:0,0:0,0:0,0:0,0:0,250:0,0:0,0:0,6398:0
5072903,NW_009646206.1,177766,rs147396508,C,T,.,.,.,AN:AC,6134:0,...,0:0,0:0,0:0,0:0,0:0,0:0,250:0,0:0,0:0,6384:0
5072924,NW_009646206.1,178042,rs148703638,C,T,.,.,.,AN:AC,6138:0,...,0:0,0:0,0:0,0:0,0:0,0:0,250:0,0:0,0:0,6388:0
5072938,NW_009646206.1,180987,rs151316795,C,T,.,.,.,AN:AC,134:0,...,0:0,48:0,0:0,0:0,0:0,0:0,188:0,48:0,0:0,370:0


In [9]:
alfa_filteredOn_rsIds_atLeast1AltAllelesNot0_df = alfa_filteredOn_rsIds_df[~alfa_filteredOn_rsIds_df.apply(is_all_alt_alleles_popu_count_zero, axis=1)]
alfa_filteredOn_rsIds_atLeast1AltAllelesNot0_df.shape

(1736207, 21)

In [10]:
alfa_filteredOn_rsIds_allAltAlleles0_df.to_csv(home_dir+"data/ALFA/alfa_filteredOn_rsIds_allAltAlleles0.tsv", index=False, sep="\t")
alfa_filteredOn_rsIds_atLeast1AltAllelesNot0_df.to_csv(home_dir+"data/ALFA/alfa_filteredOn_rsIds_atLeast1AltAllelesNot0.tsv", index=False, sep="\t")

In [13]:
snps_with_alfa_df = pd.merge(left=snps_df, right=alfa_filteredOn_rsIds_atLeast1AltAllelesNot0_df, left_on="SNP_ID", right_on="ID", how="inner")
print(snps_with_alfa_df.columns)
print(snps_with_alfa_df.shape)
print(snps_with_alfa_df["SNP_ID"].unique().shape[0])

Index(['SNP_ID', 'SNP_CLASS', 'CLINICAL_SIGNIFICANCE', 'ACC', 'CHR', 'CHRPOS',
       'SPDI', 'FXN_CLASS', 'VALIDATED', 'TAX_ID', 'CREATEDATE', 'UPDATEDATE',
       'DOCSUM', 'NP_VARIANTS', 'REF_ALLELE', 'ALT_ALLELES', 'CHROM', 'POS',
       'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMN10492695',
       'SAMN10492696', 'SAMN10492697', 'SAMN10492698', 'SAMN10492699',
       'SAMN10492700', 'SAMN10492701', 'SAMN10492702', 'SAMN11605645',
       'SAMN10492703', 'SAMN10492704', 'SAMN10492705'],
      dtype='object')
(1736207, 37)
1736207


In [14]:
print_number_of_genes_attached(snps_with_alfa_df)

#-genes: 25162


### MANE things

In [19]:
mane_mapped_genes = pd.read_csv(home_dir+"data/dbsnp/processed/mane_mapped_genes.tsv", sep="\t")
print(mane_mapped_genes.columns)
print(mane_mapped_genes.shape)

/tmp/ipykernel_2858092/3739632187.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  mane_mapped_genes = pd.read_csv(home_dir+"data/dbsnp/processed/mane_mapped_genes.tsv", sep="\t")


Index(['SNP_ID', 'GENE_NAME', 'GENE_ID', 'RefSeq_nuc', 'RefSeq_prot',
       'MANE_status', 'GRCh38_chr', 'chr_start', 'chr_end', 'chr_strand'],
      dtype='object')
(7336390, 10)


In [20]:
snps_with_alfa_mane_df = pd.merge(left=snps_with_alfa_df, right=mane_mapped_genes, on="SNP_ID", how="inner")
print(snps_with_alfa_mane_df.columns)
print(snps_with_alfa_mane_df.shape)
print(snps_with_alfa_mane_df["SNP_ID"].unique().shape)
print(snps_with_alfa_mane_df["GENE_NAME"].unique().shape)
print(snps_with_alfa_mane_df["RefSeq_prot"].unique().shape)

Index(['SNP_ID', 'SNP_CLASS', 'CLINICAL_SIGNIFICANCE', 'ACC', 'CHR', 'CHRPOS',
       'SPDI', 'FXN_CLASS', 'VALIDATED', 'TAX_ID', 'CREATEDATE', 'UPDATEDATE',
       'DOCSUM', 'NP_VARIANTS', 'REF_ALLELE', 'ALT_ALLELES', 'CHROM', 'POS',
       'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMN10492695',
       'SAMN10492696', 'SAMN10492697', 'SAMN10492698', 'SAMN10492699',
       'SAMN10492700', 'SAMN10492701', 'SAMN10492702', 'SAMN11605645',
       'SAMN10492703', 'SAMN10492704', 'SAMN10492705', 'GENE_NAME', 'GENE_ID',
       'RefSeq_nuc', 'RefSeq_prot', 'MANE_status', 'GRCh38_chr', 'chr_start',
       'chr_end', 'chr_strand'],
      dtype='object')
(1890003, 46)
(1718358,)
(18940,)
(18998,)


In [25]:
temp_merged_df = pd.merge(left=snps_with_alfa_mane_df, right=snp_genes_df, on=["SNP_ID", "GENE_NAME"], how="inner")
print("#-genes:", temp_merged_df["GENE_NAME"].unique().shape[0])

#-genes: 18940


In [27]:
def filter_variants_on_canonical_proteins(row):
    variations = row["NP_VARIANTS"].split(",")
    new_variations = []
    for var in variations:
        if var.split(":")[0] == row["RefSeq_prot"]:
            new_variations.append(var)
    return ",".join(new_variations)

snps_with_alfa_mane_df["mane_protein_variants"] = snps_with_alfa_mane_df[["NP_VARIANTS", "RefSeq_prot"]].apply(filter_variants_on_canonical_proteins, axis=1)
print(snps_with_alfa_mane_df.shape)

from utils.pandas_extented_filters import filter_remove_null_nan_empty_entries
mane_filtered_snps_df = filter_remove_null_nan_empty_entries(snps_with_alfa_mane_df, a_col_name="mane_protein_variants") # NNE: null, nan or empty variants
print(mane_filtered_snps_df.shape)
print(mane_filtered_snps_df["SNP_ID"].unique().shape[0])
print(mane_filtered_snps_df["GENE_NAME"].unique().shape[0])
print(mane_filtered_snps_df["RefSeq_prot"].unique().shape[0])

(1890003, 47)
Number of NAN rows removed: 0
Number of NULL rows removed: 0
Number of empty rows removed: 204160
(1685843, 47)
1673973
18990
18932


In [28]:
mane_filtered_snps_df.to_csv(home_dir+"data/temp_popufreq_dataset/snps_with_alfa_mane_raw.tsv", sep="\t", index=False) # 2.4GB